## Beam conversion from Bigquery to TF Records

In this notebook we use Apache Beam to convert to tfrecords
The applications can be found in `beam_candidates` and `beam_training` for candidate generation and training

`pip install --upgrade 'apache-beam[gcp]'`

#### IMPORTANT - make sure you upgrade Dataflow with the above command then restart the notebook

In [ ]:
# !pip install --upgrade 'apache-beam[gcp]'

In [2]:
import tensorflow as tf

In [3]:
# ! gsutil mb -l us-central1 gs://spotify-beam-v3

In [4]:
# setup
PROJECT_ID = 'hybrid-vertex'
BUCKET_NAME = 'spotify-beam-v3' # 'spotify-tfrecords-blog' # Set your Bucket name
REGION = 'us-central1' # Set the region for Dataflow jobs
VERSION = 'v6'

### Run the Dataflow app to convert from BQ to TFrecords

Candidate generation can be found in `beam_candidates`
Training and Validation generation can be found in `beam_training`

Usage:

  Candidate generation 
  
  `beam_candidates\python3 main.py`
   
  Training generation
  
  `beam_training\python3 main-train.py <BQ_table> <gcs data subfolder> <desired partition size MB> <BQ dataset size MB> <version tag>`
  
  
##### Be careful with quotas - running more than two jobs can run into quota issues with defaults

Training data generation runs about 1 hour with 10 workers

In [6]:
%cd beam_training

/home/jupyter/spotify_mpd_two_tower/beam_training


In [ ]:
! python3 main-train.py train_flatten train 800 675_990 $VERSION

In [ ]:
! python3 main-train.py train_flatten_valid valid 800 6_800 $VERSION

#### Generating the different artist tfrecords

In [ ]:
! python3 main-train.py train_flatten_dif_artist_valid dif_artist_valid 800 4_930 $VERSION

In [1]:
! python3 main-train.py train_flatten_dif_artist dif_artist 800 488_680 $VERSION

python3: can't open file 'main-train.py': [Errno 2] No such file or directory


In [8]:
! python3 main-train.py train_flatten_last_5 train_last_5 800 88_940 $VERSION

Number of Expected TFRecords: 111
GoogleCloudOptions(create_from_snapshot=None, dataflow_endpoint=https://dataflow.googleapis.com, dataflow_kms_key=None, dataflow_service_options=None, enable_artifact_caching=False, enable_hot_key_logging=False, enable_streaming_engine=False, flexrs_goal=None, impersonate_service_account=None, job_name=spotify-bq-tfrecords-v6-221005-155712, labels=None, no_auth=False, project=hybrid-vertex, region=us-central1, service_account_email=None, staging_location=gs://spotify-beam-v3/v6/job/staging/, temp_location=gs://spotify-beam-v3/v6/job/temp/, template_location=None, transform_name_mapping=None, update=False)
/opt/conda/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:2739: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = pcoll.pipeline.options.view_as(



^C
Traceback (most recent call last):
  File "main-train.py", line 79, in <module>
    main()
 

In [ ]:
! python3 main-train.py train_flatten_valid_last_5 valid_last_5 800 920 $VERSION

Number of Expected TFRecords: 1
GoogleCloudOptions(create_from_snapshot=None, dataflow_endpoint=https://dataflow.googleapis.com, dataflow_kms_key=None, dataflow_service_options=None, enable_artifact_caching=False, enable_hot_key_logging=False, enable_streaming_engine=False, flexrs_goal=None, impersonate_service_account=None, job_name=spotify-bq-tfrecords-v6-221005-144245, labels=None, no_auth=False, project=hybrid-vertex, region=us-central1, service_account_email=None, staging_location=gs://spotify-beam-v3/v6/job/staging/, temp_location=gs://spotify-beam-v3/v6/job/temp/, template_location=None, transform_name_mapping=None, update=False)
/opt/conda/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:2739: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = pcoll.pipeline.options.view_as(





## Test the output

In [ ]:
## testing output
from google.cloud import storage

client = storage.Client()
# # gs://spotify-beam-v3/v3/candidates/*.tfrecords

BUCKET = 'spotify-beam-v3'
CANDIDATE_PREFIX = 'v3/candidates/'

candidate_files = []
for blob in client.list_blobs(f"{BUCKET}", prefix=f'{CANDIDATE_PREFIX}', delimiter="/"):
    candidate_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))
    
candidate_dataset = tf.data.TFRecordDataset(candidate_files)

def parse_candidate_tfrecord_fn(example):
    example = tf.io.parse_single_example(
        example, 
        features=candidate_features
    )
    return example

# parsed_candidate_dataset = candidate_dataset.map(parse_candidate_tfrecord_fn, num_parallel_calls=-1)

parsed_candidate_dataset = candidate_dataset.map(parse_candidate_tfrecord_fn) ### THIS NEEDS TO BE FIXED SO THE UNIQUE PRODUCT DATASET HAS THE SAME FIELD NAMES (goes thru the same model)

In [ ]:
for x in parsed_candidate_dataset.batch(2).take(1):
    print(x)

In [ ]:
all_features = {
    'track_name_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'artist_name_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'album_name_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'track_uri_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'artist_uri_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'album_uri_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'duration_ms_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'track_pop_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'artist_pop_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'artist_genres_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'artist_followers_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    # 'pos_seed_track': tf.io.FixedLenFeature(dtype=tf.int64, shape=()),
    'track_name_seed_track': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'artist_name_seed_track': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'album_name_seed_track': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'track_uri_seed_track': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'artist_uri_seed_track': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'album_uri_seed_track': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'duration_seed_track': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'track_pop_seed_track': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'artist_pop_seed_track': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'artist_genres_seed_track': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'artist_followers_seed_track': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    # 'pid': tf.io.FixedLenFeature(dtype=tf.int64, shape=()),
    'name': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'collaborative': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    # 'duration_ms_seed_pl': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'n_songs_pl': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'num_artists_pl': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'num_albums_pl': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'description_pl': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    ###ragged
    'track_name_pl': tf.io.RaggedFeature(tf.string),
    'artist_name_pl': tf.io.RaggedFeature(tf.string),
    'album_name_pl': tf.io.RaggedFeature(tf.string),
    'track_uri_pl': tf.io.RaggedFeature(tf.string),
    'duration_ms_songs_pl': tf.io.RaggedFeature(tf.float32),
    'artist_pop_pl': tf.io.RaggedFeature(tf.float32),
    'artists_followers_pl': tf.io.RaggedFeature(tf.float32),
    'track_pop_pl': tf.io.RaggedFeature(tf.float32),
    'artist_genres_pl': tf.io.RaggedFeature(tf.string),
}


In [ ]:
## testing output
from google.cloud import storage

client = storage.Client()
# # gs://spotify-beam-v3/v3/candidates/*.tfrecords

BUCKET = 'spotify-beam-v3'
CANDIDATE_PREFIX = 'v3/valid/'

valid_files = []
for blob in client.list_blobs(f"{BUCKET}", prefix=f'{CANDIDATE_PREFIX}', delimiter="/"):
    valid_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))
    
valid = tf.data.TFRecordDataset(valid_files)

def parse_tfrecord(example):
    example = tf.io.parse_single_example(
        example, 
        features=all_features
    )
    return example

# parsed_candidate_dataset = candidate_dataset.map(parse_candidate_tfrecord_fn, num_parallel_calls=-1)

valid_parsed = valid.map(parse_tfrecord) ### THIS NEEDS TO BE FIXED SO THE UNIQUE PRODUCT DATASET HAS THE SAME FIELD NAMES (goes thru the same model)

In [ ]:
for x in valid_parsed.batch(2).take(1):
    print(x)